# Codice

In [1]:
%pip install -q "nbformat==5.10.3"

In [2]:
%pip install -q "ipywidgets"

In [3]:
%pip install -q "anywidget"

In [4]:
%pip install -q "plotly"

In [24]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import asyncio

# Function to rotate point
def rotate_point(x, y, z, theta):
    x_new = x * np.cos(theta) + z * np.sin(theta)
    y_new = y
    z_new = -x * np.sin(theta) + z * np.cos(theta)
    return x_new, y_new, z_new

# Function to rotate arrow
def rotate_arrow(x_wav, z_wav, x_orig, y_orig, z_orig):
    theta = np.arctan2(x_wav, z_wav)  
    x_rot, y_rot, z_rot = [], [], []
    
    for xx, yy, zz in zip(x_orig, y_orig, z_orig):
        if xx is None or yy is None or zz is None:
            x_rot.append(None)
            y_rot.append(None)
            z_rot.append(None)
        else:
            xr, yr, zr = rotate_point(xx, yy, zz, theta)
            x_rot.append(xr + x_wav)
            y_rot.append(yr)
            z_rot.append(zr + z_wav)

    return x_rot, y_rot, z_rot

# Constants
Darrow = 5e-2
y0 = 5e-2
x_orig = [0, 0, None, Darrow, 0, -Darrow]
y_orig = [y0, y0, None, y0, y0, y0]
z_orig = [0, 0., None, -Darrow, 0., -Darrow]

# Create figure widget
fig = go.FigureWidget(layout=dict(width=1000, height=800))

# Parameters
num_segments = 50
y = np.linspace(0, 2 * np.pi, 200)
segment_indices = np.linspace(0, len(y)-1, num_segments, dtype=int)
k = 5

# Initialize wave parameters
Ex = 1.
Ez = 0.
phase_deg = 0
phase = np.radians(phase_deg)

# Compute initial wave
x_wave = Ex * np.sin(k * y)
z_wave = Ez * np.sin(k * y + phase)

# Add main sine wave
fig.add_trace(go.Scatter3d(x=x_wave, y=y, z=z_wave, mode='lines', line=dict(width=5, color='blue')))

# Add connecting segment traces (projections)
for idx in segment_indices:
    fig.add_trace(go.Scatter3d(
        x=[x_wave[idx], 0, None],
        y=[y[idx], y[idx], None],
        z=[z_wave[idx], 0, None],
        mode='lines',
        line=dict(color='gray', width=2),
        showlegend=False
    ))

# Add vector components
fig.add_trace(go.Scatter3d(x=[0, 0, None, Darrow, 0., -Darrow], 
                           y=[y0, y0, None, y0, y0, y0], 
                           z=[z_wave[0], 0, None, z_wave[0]-0.05*np.sign(z_wave[0]), z_wave[0], z_wave[0]-0.05*np.sign(z_wave[0])], 
                           mode='lines', line=dict(color='red', width=8), showlegend=False))  
fig.add_trace(go.Scatter3d(x=[x_wave[0], 0, None, x_wave[0]-0.05*np.sign(x_wave[0]), x_wave[0], x_wave[0]-0.05*np.sign(x_wave[0])], 
                           y=[y0, y0, None, y0, y0, y0], 
                           z=[0, 0, None, Darrow, 0., -Darrow], 
                           mode='lines', line=dict(color='green', width=8), showlegend=False))  
fig.add_trace(go.Scatter3d(x=[x_wave[0], 0, None], 
                           y=[y0, y0, None], 
                           z=[z_wave[0], 0, None], 
                           mode='lines', line=dict(color='blue', width=8), showlegend=False))  

x_rot, y_rot, z_rot = rotate_arrow(x_wave[0], z_wave[0], x_orig, y_orig, z_orig)
fig.add_trace(go.Scatter3d(x=x_rot, y=y_rot, z=z_rot, mode='lines', line=dict(color='blue', width=8), showlegend=False))

fig.add_trace(go.Scatter3d(x=[-20], y=[-.1], z=[0.3], mode='text', text=['Ex'], textposition='middle right', textfont=dict(color='green',size=28), showlegend=False))
fig.add_trace(go.Scatter3d(x=[0.3], y=[-.1], z=[-20], mode='text', text=['Ez'], textposition='middle right', textfont=dict(color='red',size=28), showlegend=False))

fig.add_trace(go.Scatter3d(x=[-1.5, 1.5, None, 1.4, 1.5, 1.4], y=[0, 0, None, 0, 0, 0], z=[0, 0, None, 0.1, 0, -0.1], mode='lines', line=dict(color='black', width=4), name='X-axis'))
fig.add_trace(go.Scatter3d(x=[0, 0, None, 0.1, 0, -0.1], y=[-0.5, 6.5, None, 6.4, 6.5, 6.4], z=[0, 0, None, 0, 0, 0], mode='lines', line=dict(color='black', width=4), name='Y-axis'))
fig.add_trace(go.Scatter3d(x=[0, 0, None, 0.1, 0, -0.1], y=[0, 0, None, 0, 0, 0], z=[-1.5, 1.5, None, 1.4, 1.5, 1.4], mode='lines', line=dict(color='black', width=4), name='Z-axis'))

# Axis labels
fig.add_trace(go.Scatter3d(x=[1.8], y=[0], z=[0], mode='text', text=['X'], textposition='middle right', showlegend=False))
fig.add_trace(go.Scatter3d(x=[0], y=[6.5], z=[0], mode='text', text=['Y'], textposition='middle right', showlegend=False))
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[1.6], mode='text', text=['Z'], textposition='middle right', showlegend=False))


fig.update_layout(
    showlegend=False,
    scene=dict(
        xaxis=dict(range=[-1.7, 1.8], showticklabels=False, title=''),
        yaxis=dict(range=[-1, 7], showticklabels=False, title=''),
        zaxis=dict(range=[-1.5, 1.8], showticklabels=False, title=''),
        aspectratio=dict(x=1, y=2, z=1),
    )
)

# Function to update traces dynamically
def update_plot(Ex=1, Ez=0, phase_deg=0, frame=0):
    phase = np.radians(phase_deg)
    x_wave = Ex * np.sin(k * y - frame / 10.0)
    z_wave = Ez * np.sin(k * y - frame / 10.0 + phase)

    with fig.batch_update():
        fig.data[0].x = x_wave
        fig.data[0].z = z_wave
        
        for i, idx in enumerate(segment_indices):
            fig.data[i + 1].x = [x_wave[idx], 0, None]
            fig.data[i + 1].z = [z_wave[idx], 0, None]

        x_rot, y_rot, z_rot = rotate_arrow(x_wave[0], z_wave[0], x_orig, y_orig, z_orig)

        if x_wave[0]==0:
            fig.data[-8].text=['']
        else:
            fig.data[-8].text=['Ex']
        fig.data[-8].x = [x_wave[0]]
        
        if z_wave[0]==0:
            fig.data[-7].text=['']
        else:
            fig.data[-7].text=['Ez']
        fig.data[-7].z = [z_wave[0]]
        
        fig.data[-9].x = x_rot
        fig.data[-9].z = z_rot
        fig.data[-10].x = [x_wave[0], 0, None]
        fig.data[-10].z = [z_wave[0], 0, None]
        
        fig.data[-11].x = [x_wave[0], 0, None, x_wave[0] - 0.05 * np.sign(x_wave[0]), x_wave[0], x_wave[0] - 0.05 * np.sign(x_wave[0])]
        fig.data[-12].z = [z_wave[0], 0, None, z_wave[0] - 0.05 * np.sign(z_wave[0]), z_wave[0], z_wave[0] - 0.05 * np.sign(z_wave[0])]

# Call update once to initialize
update_plot()

# Sliders
Ex_slider = widgets.FloatSlider(value=1, min=-1, max=1, step=.1, description="E_x0")
Ez_slider = widgets.FloatSlider(value=0, min=-1, max=1, step=.1, description="E_z0")
phase_slider = widgets.FloatSlider(value=0, min=-180, max=180, step=1, description="Phase (°)")

# Animation function (Async)
running = False

async def animate():
    global running
    frame = 0
    while running:
        update_plot(Ex_slider.value, Ez_slider.value, phase_slider.value, frame)
        frame += 1
        await asyncio.sleep(0.05)

def start_animation(_):
    global running
    if not running:
        running = True
        asyncio.create_task(animate())

def stop_animation(_):
    global running
    running = False

# Buttons
start_button = widgets.Button(description="Start", button_style='success')
stop_button = widgets.Button(description="Stop", button_style='danger')

start_button.on_click(start_animation)
stop_button.on_click(stop_animation)

# Display
interactive_figure = widgets.VBox([Ex_slider, Ez_slider, phase_slider, widgets.HBox([start_button, stop_button])])


# Polarizzazione

### Descrizione
Il notebook dimostra la polarizzazione di un'onda armonica che si propaga lungo l'asse $y$.

È possibile variare l'ampiezza delle componenti $E_x$ ed $E_z$ del campo elettrico ed la loro fase relativa.

In [25]:
display(fig, interactive_figure)

FigureWidget({
    'data': [{'line': {'color': 'blue', 'width': 5},
              'mode': 'lines',
              'type': 'scatter3d',
              'uid': 'fd79e248-e0b2-46ec-83e4-4b55a7c4c033',
              'x': {'bdata': ('AAAAAAAAAAB+FYL6lh/EPxrx7QWH39' ... 'WH39O/ixWC+pYfxL8Jc9m/rw/WvA=='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAAAAAABMJ0jGcCqgP0wnSMZwKr' ... 'FR4RhAypy3cqYBGUAYLURU+yEZQA=='),
                    'dtype': 'f8'},
              'z': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAACAAAAAAAAAAIAAAAAAAAAAgA=='),
                    'dtype': 'f8'}},
             {'line': {'color': 'gray', 'width': 2},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '37291107-2b23-404f-906b-efb369acdb0d',
              'x': [0.0, 0, None],
              'y': [0.0, 0.0, None],
              'z': [0.0, 0, None]},
             {'line': {'color': 'gray', 'width': 2},
              